In [1]:

import osmnx as ox
import geopandas as gpd
import requests
import osm2geojson


def get_osm_data(tag_query, extent, area_type):
    osm_crs = 'EPSG:4326'
    url = "http://overpass-api.de/api/interpreter"

    my_crs = extent.crs
    if extent.crs != osm_crs:
        extent = extent.to_crs(osm_crs)

    if area_type == 'bbox':
        bbox = extent.total_bounds
        area = '{1},{0},{3},{2}'.format(bbox[0], bbox[1], bbox[2], bbox[3])
    elif area_type == 'convexhull':
        xy = extent.convex_hull.geometry[0].exterior.coords.xy
        osm_poly = 'poly:"'
        for i, (lat, lon) in enumerate(zip(xy[1], xy[0])):
            osm_poly = osm_poly + '{} {} '.format(lat, lon)
        area = osm_poly[:-1] + '"'
    else:
        print('Unknown type {}: please use [bbox] or [convexhull]'.format(area_type))
        return None

    data_request = """
        [out:json];
        (
            node{0}({1});
            way{0}({1});
            relation{0}({1});
        );
        out geom;
        >;
        out skel qt;
    """.format(tag_query, area)

    response = requests.get(url, params={'data': data_request})
    geojson = osm2geojson.json2geojson(response.json())
    gdf = gpd.GeoDataFrame.from_features(geojson)
    gdf = gdf[gdf['tags'].notna()]  # to avoid including both ways and their waypoints
    gdf.crs = osm_crs
    gdf = gdf.to_crs(my_crs)  # CRS of our input dataset
    return gdf

In [2]:
area_type = 'convexhull'
extent = ox.geocode_to_gdf('Amsterdam')

# get parks
park_query = '["leisure" = "park"]'

# get squares (1)
square_query = '["place" = "square"]'

# get squares (2)
plein_query = '["name" ~ "plein"]["name"!~"buurt"]["name"!~"Bus "]'

#other efforts to get squares (not good)
# tag_query = '["highway" = "pedestrian"]["area"="yes"]'

#get marketplaces
# tag_query = '["amenity" = "marketplace"]'

In [3]:
gdf_park = get_osm_data(park_query, extent, area_type)
gdf_square = get_osm_data(square_query, extent, area_type)
gdf_plein = get_osm_data(plein_query, extent, area_type)


# gdf.to_file("../data/ams_namepleinnotbuurt_osm.geojson", driver="GeoJSON")
# print(gdf_park.head())
# fig, ax = plt.subplots(1, 1)
# gdf_park.plot(ax=ax)
# gdf_square.plot(ax=ax)
# plt.show()

In [4]:
gdf_park.head()

,geometry,type,id,tags
0,POINT (4.89031 52.33209),node,4609693622,"{'addr:street': 'Europaboulevard', 'leisure': ..."
1,POINT (4.88043 52.38173),node,7396128924,"{'leisure': 'park', 'name': 'Eerste Marnixplan..."
2,POINT (4.87608 52.37459),node,7396143627,"{'leisure': 'park', 'name': 'Tweede Marnixplan..."
3,"POLYGON ((4.91667 52.35941, 4.91608 52.36021, ...",way,4366722,"{'leisure': 'park', 'name': 'Oosterpark', 'wik..."
4,"POLYGON ((4.90889 52.36833, 4.90818 52.36784, ...",way,4376560,"{'leisure': 'park', 'name': 'Wertheimpark', 'w..."


In [22]:
gdf_plein.shape

(2420, 4)

In [26]:
gdf_plein = gdf_plein.loc[(gdf_plein.geometry.geom_type=='Polygon') | (gdf_plein.geometry.geom_type=='MultiPolygon')]
gdf_park = gdf_park.loc[(gdf_park.geometry.geom_type=='Polygon') | (gdf_park.geometry.geom_type=='MultiPolygon')]
gdf_square = gdf_square.loc[(gdf_square.geometry.geom_type=='Polygon') | (gdf_square.geometry.geom_type=='MultiPolygon')]

In [25]:
gdf_plein.loc[(gdf_plein.geometry.geom_type=='Polygon') | (gdf_plein.geometry.geom_type=='MultiPolygon')].shape

(238, 4)

In [27]:
gdf_plein.shape

(238, 4)

In [28]:
gdf_square.shape

(57, 4)

In [30]:
import pandas
gdf =   pandas.concat([gdf_plein, gdf_square])

In [35]:
gdf.shape

(295, 4)

In [34]:
gdf_park.shape

(290, 4)

In [36]:
gdf = gpd.read_file('pop_ams_all.csv')
gdf.crs = 'epsg:4326'        

SyntaxError: invalid syntax (<ipython-input-36-32c1f3b5c161>, line 1)